In [1]:
import lmdb
import torch
from ocpmodels.datasets import SinglePointLmdbDataset
import pandas as pd
import pickle
import gc

In [2]:
#for train sample
dataset_size_list = {
    0: "10k",
    1: "100k",
    2: "all"
}
mode_list = {
    0: "train",
    1: "val",
    2: "test"
}
dataset_list = {
    1: "id",
    2: "ood_ads",
    3: "ood_cat",
    4: "ood_both"
}

# setting section
root = "../../ocp_datasets/data/is2re"
dataset_size = dataset_size_list[0]
mode = mode_list[0]
dataset = dataset_list[4]
#

def path_build(root, dataset_size, mode, dataset): # : mode: "origin, origin_old, target"
    path = f'{root}/{dataset_size}/{mode}'
    if mode != mode_list[0]:
        path = f'{path}_{dataset}'
    return path
    
dataset_origin_path = f'{path_build(root, dataset_size, mode, dataset)}/structures.pkl'
dataset_origin_old_path = f'{path_build(root, dataset_size, mode, dataset)}/data.lmdb'
dataset_target_path = f'{path_build(root, dataset_size, mode, dataset)}/data_mod.lmdb'

(print(
    f'dataset_origin_path: {dataset_origin_path}',
    f'dataset_origin_old_path: {dataset_origin_old_path}',
    f'dataset_target_path: {dataset_target_path}',
    sep='\n')
)
#/home/alex/Documents/ocp_datasets/data/is2re/all/val_ood_both
#/home/alex/Documents/ocp_datasets/data/is2re/all/test_ood_both


dataset_origin_path: ../../ocp_datasets/data/is2re/10k/train/structures.pkl
dataset_origin_old_path: ../../ocp_datasets/data/is2re/10k/train/data.lmdb
dataset_target_path: ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb


In [3]:
# dataset_origin_old = SinglePointLmdbDataset({"src": dataset_origin_old_path})

In [4]:
# dataset_origin_old[0]['distances']

In [5]:
# dataset_origin = pd.read_pickle(dataset_origin_path)

In [6]:
# dataset_origin[0]['distances_new']

In [7]:
def update_dataset(dataset_target_path, dataset_origin_old_path, dataset_origin_path, features_names=None):
    dataset_origin = pd.read_pickle(dataset_origin_path)
    
    dataset_origin_old = SinglePointLmdbDataset({"src": dataset_origin_old_path})
    
    dataset_target = lmdb.open(
        dataset_target_path,
        map_size=int(1e9*5), #~ 5 Gbyte
        subdir=False,
        meminit=False,
        map_async=True,
    )

    idx = 0

    for ii, data_object_origin_old in enumerate(dataset_origin_old):

            # Substitute: edge_index -> edge_index_new
            data_object = dataset_origin_old[ii]
            for feature_name in features_names:
                feature = torch.from_numpy(dataset_origin[ii][feature_name+'_new'])
                data_object[feature_name] = feature

            # Write to LMDB
            txn = dataset_target.begin(write=True)
            txn.put(f"{idx}".encode("ascii"), pickle.dumps(data_object, protocol=-1))
            txn.commit()
            dataset_target.sync()
            if idx % 1000 == 0:
                print('{} of {} for file {}'.format(idx, len(dataset_origin_old), dataset_target_path))
            idx += 1

    dataset_target.close()
    print("done")

In [9]:
update_dataset(dataset_target_path, dataset_origin_old_path, dataset_origin_path, features_names=['edge_index', 'distances'])

0 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
1000 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
2000 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
3000 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
4000 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
5000 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
6000 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
7000 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
8000 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
9000 of 10000 for file ../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb
done


In [9]:
dataset_target = SinglePointLmdbDataset({"src": dataset_target_path})

In [10]:
dataset_target[0]['distances']

tensor([ 2.4172,  2.4172,  2.2962,  ..., 19.1239, 19.2233, 19.2233],
       dtype=torch.float64)